# 线性回归的简洁实现

In [18]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = d2l.synthetic_data(true_w,true_b,1000)

print(true_w)
features,labels #features每一行都包含一个二维数据样本，labels中每一行都包含一维标签值

tensor([ 2.0000, -3.4000])


(tensor([[-1.5649,  0.7203],
         [ 1.5154, -1.5000],
         [-1.0931, -0.0290],
         ...,
         [-0.2199, -1.1979],
         [-0.0246, -0.0533],
         [-0.7267, -0.5636]]),
 tensor([[-1.3754e+00],
         [ 1.2352e+01],
         [ 2.1159e+00],
         [ 8.6322e+00],
         [ 9.4046e+00],
         [ 2.5230e+00],
         [-8.7431e+00],
         [ 5.8420e+00],
         [ 7.9634e+00],
         [ 3.3649e+00],
         [ 6.9112e+00],
         [ 5.2827e+00],
         [ 6.1146e+00],
         [ 1.1655e+00],
         [ 4.3201e+00],
         [-5.1092e-02],
         [ 3.7814e+00],
         [ 4.1781e+00],
         [ 2.4285e+00],
         [-7.8458e+00],
         [ 5.6785e+00],
         [ 8.8036e+00],
         [ 8.0877e+00],
         [ 3.2861e+00],
         [ 7.0725e+00],
         [ 6.3880e+00],
         [ 2.1396e+00],
         [ 5.1197e+00],
         [ 6.7331e-01],
         [ 1.7744e+00],
         [-2.4101e+00],
         [ 5.8419e+00],
         [-3.3526e-01],
         [ 5.7177e

In [19]:
def load_array(data_arrays,batch_size,is_train=True): #@save
    '''构建一个pytorch数据迭代器'''
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size = 10
data_iter = load_array((features,labels),batch_size)

In [20]:
next(iter(data_iter))

[tensor([[-0.3707,  0.4474],
         [-0.7459, -0.8924],
         [-2.1110,  2.5604],
         [ 1.1059, -0.2765],
         [ 0.6713, -0.7656],
         [-0.3554,  0.6703],
         [ 0.2339,  0.3444],
         [ 0.9735,  0.9804],
         [-0.1275,  0.8418],
         [ 1.0584,  1.5848]]),
 tensor([[ 1.9503],
         [ 5.7539],
         [-8.7431],
         [ 7.3447],
         [ 8.1411],
         [ 1.2055],
         [ 3.4888],
         [ 2.8199],
         [ 1.0746],
         [ 0.9282]])]

## 定义模型
只需要关注使用哪些层来构造模型，而不必关注层的实现细节。
第一个参数指定输入特征形状，第二个指定输出特征形状，为单个标量，因此为1。

In [21]:
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

## 初始化模型参数

通过net[0]来选择网络中的第一个图层，然后使用weight.data和bias.data方法访问参数，我们还可以使用替换方法normal_和fill_来重写参数值。

In [22]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数
计算均方误差使用的是MSELoss,也成平方$L_2$范数,默认情况下，它返回所有样本损失的平均值。

In [23]:
loss = nn.MSELoss()

## 定义优化算法

我们需要指定优化参数，可以通过net.parameters()从我们的模型中获得，以及优化算法所需的超参数字典

In [24]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

## 训练
在每个迭代周期里，我们将完整的遍历一次数据集，不停从中获取一个小批量的输入和相应的标签。对于每个小批量，我们会进行以下步骤：
- 通过调用net(X)生成预测并计算损失l(前向传播)。
- 通过进行反向传播来计算梯度。
- 通过调用优化器来更新模型参数。

In [25]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch+1},loss {l:f}')

epoch 1,loss 0.000232
epoch 2,loss 0.000098
epoch 3,loss 0.000098


In [26]:
w = net[0].weight.data
print('w的估计误差: ',true_b - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差: ',true_b - b)

w的估计误差:  tensor([2.2005, 7.6004])
b的估计误差:  tensor([0.0007])
